In [62]:
import json
import automata
import bisect
from matcher import Matcher

In [71]:
with open('output/distance_result.json','r') as out:
    for line in out:
        data = json.loads(line)

In [85]:
filterTrue = [d for d in data if d['is_correct'] == True]
filterFalse = [d for d in data if d['is_correct'] == False]

In [86]:
print(len(data),len(filterTrue), len(filterFalse))

(401, 304, 97)


In [92]:
with open('output/automaton_result.json','r') as out:
    for line in out:
        automaton_data = json.loads(line)

In [93]:
filterTrue2 = [d for d in automaton_data if d['is_match'] == True]
filterFalse2 = [d for d in automaton_data if d['is_match'] == False]

In [94]:
print(len(data),len(filterTrue2), len(filterFalse2))

(401, 315, 86)


In [89]:
with open('output/distance_result_2.json','r') as out:
    for line in out:
        data_2 = json.loads(line)

In [95]:
filterTrue3 = [d for d in data_2 if d['is_correct'] == True]
filterFalse3 = [d for d in data_2 if d['is_correct'] == False]

In [91]:
print(len(data_2),len(filterTrue3), len(filterFalse3))

(401, 307, 94)
